In [32]:
import pandas as pd
import re
import glob
from datetime import datetime
import nltk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']= (17,5)
from collections import Counter
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
from tqdm import tqdm
from nltk.corpus import PlaintextCorpusReader


In [33]:
#nltk.download("punkt")
database = [ ]
corpus = PlaintextCorpusReader("./DATA", ".*\.txt")
files=corpus.fileids()
for f in files:
    number_of_sentences = len(corpus.sents(f))
    number_of_words = len(
        [word for sentence in corpus.sents(f) for word in sentence])
    number_of_fids = len(corpus.fileids())
    path = "./DATA/"
    newstr = "".join((path, f))
    with open(newstr, encoding= 'Latin') as speech_file:
        number = str(''.join(filter(str.isdigit, f)))
        f = f.replace(".txt", "")
        debater = ''.join(filter(lambda x: not x.isdigit(), f))
        speeches = {
            'filename' : f,
            'content' : speech_file.read(),
            'debater' : debater,
            'debate_number' : number,
            'number_of_sentences' : number_of_sentences,
            'number_of_words' : number_of_words,
            'average_sentence_length': number_of_words/number_of_sentences
        }   
    database.append(speeches)
df = pd.DataFrame(database)
df.head()
%store df

Stored 'df' (DataFrame)


In [34]:
df.head()

,filename,content,debater,debate_number,number_of_sentences,number_of_words,average_sentence_length
0,Arraras22,"""ARRARÃS: Dr. Carson, concerning this recomme...",Arraras,22,32,684,21.375000
1,Bachmann138,"""BACHMANN: Well, it's very clear in the last f...",Bachmann,138,118,2416,20.474576
2,Bachmann145,"""BACHMANN: Well-- one of our former competitor...",Bachmann,145,154,2919,18.954545
3,Bachmann151,"""BACHMANN: Good to see you, Wolf. [applause]"",...",Bachmann,151,123,2162,17.577236
4,Bachmann159,"""BACHMANN: They absolutely have, but it's unfo...",Bachmann,159,89,1565,17.584270


In [35]:
#WEB APP
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_building_blocks as dbb
import plotly.express as px
import plotly.subplots as sp


%store -r df
#import quandl

class Graph(dbb.Block):

    def layout(self):
        return html.Div([
            dcc.Dropdown(
                id=self.register('dropdown'),
                options=self.data.options,
                value=self.data.value
            ),
            dcc.Graph(id=self.register('graph'))
        ], style={'width': '500'})

    def callbacks(self):
        @self.app.callback(
            self.output('graph', 'figure'),
            [self.input('dropdown', 'value')]
        )
        def update_charts(Debator):
            figure1_traces = []
            figure2_traces = []
            filtered_data = df[df["debator"] == Debator] #the graph/dataframe will be filterd by "Debator"
            scatter = px.scatter(
                    filtered_data,
                    x="average_sentence_length",
                    y="debator",
                    size="number_of_sentences",
                    color="number_of_sentences",
                    color_continuous_scale=px.colors.sequential.Plotly3,
                    title="Something funny test 2",
            )
            scatter.update_layout(
                width =500, 
                xaxis_tickangle=30,
                title=dict(x=0.5),
                xaxis_tickfont=dict(size=9),
                yaxis_tickfont=dict(size=9),
                margin=dict(l=500, r=20, t=50, b=20),
                paper_bgcolor="LightSteelblue",
            )
            for trace in range(len(scatter["data"])):
                figure1_traces.append(scatter["data"][trace])
            
            bar = px.bar(
                filtered_data,
                x=filtered_data.groupby("debator")["number_of_words"].agg(sum),
                y=filtered_data["debator"].unique(),
                color=filtered_data.groupby("debator")["number_of_words"].agg(sum),
                color_continuous_scale=px.colors.sequential.RdBu,
                text=filtered_data.groupby("debator")["number_of_words"].agg(sum),
                title="Something funny test 1",
                orientation="h",
            )
            bar.update_layout(
                title=dict(x=0.5), margin=dict(l=550, r=20, t=60, b=20), paper_bgcolor="#D6EAF8"
            )
            bar.update_traces(texttemplate="%{text:.2s}")
            for trace in range(len(bar["data"])):
                figure2_traces.append(bar["data"][trace])
            
            this_figure = sp.make_subplots(rows=2, cols=1) 
            for traces in figure1_traces:
                this_figure.append_trace(traces, row=1, col=1)
            for traces in figure2_traces:
                this_figure.append_trace(traces, row=2, col=1)
            this_figure.update_layout(
                width =800,
                height=600)
            return this_figure
        
        

app = JupyterDash(__name__, external_stylesheets=[
    "https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap-grid.min.css"
])

options = [
            {'label': debator, 'value':debator}
            for debator in df.debater.unique()
          ]

data = {
    'options': options,
    'value': 'Biden'
}

n_graphs = 2
graphs = [Graph(app, data) for _ in range(n_graphs)]
print(graphs)

app.layout = html.Div(
    [html.Div(graph.layout, className='container')
    for graph in graphs],
    className='six.columns',
    style = {'display' : 'flex'}
)


for graph in graphs:
    graph.callbacks()

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})
if __name__ == '__main__':
    app.run_server(mode="Inline", debug=True, port=8004)


[<__main__.Graph object at 0x7fad6b69a280>, <__main__.Graph object at 0x7fad69d5ca90>]


/root/venv1/lib/python3.8/site-packages/dash/resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

